如何加载和存储权重向量和整个模型。

## 1、加载和保存张量

对于单个张量，直接调用load和save进行读写。

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
x = torch.arange(4)

In [4]:
torch.save(x, 'x-file')

In [5]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [6]:
# 存储一个张量列表
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [7]:
# 写入或读取从字符串映射到张量的字典。
# 当我们要读取或写入模型中的所有权重时，这很方便。
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

## 2、加载和保存模型参数
深度学习框架提供了内置函数来保存和加载整个网络。

注意：保存模型的参数而不是保存整个模型。为了恢复模型，我们需要用代码生成结构，然后从磁盘加载参数。

In [8]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
    
    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

In [9]:
net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

将模型的参数存储为一个叫做“mlp.params”的文件。

In [11]:
torch.save(net.state_dict(), 'mlp.params')

实例化了原始多层感知机模型的一个备份,读取参数恢复模型。

In [12]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [13]:
# 验证输入相同X, 结果一样
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])